In [ ]:
import pickle
import random
import time, os
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from fake_useragent import UserAgent
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException

The function below returns a list of links for each phone on "91mobile.com/phonefinder.php" from the 4g and 5g category

In [ ]:
def get_phones_links_4g_5g():
    chromedriver = r"C:\Program Files\Google\Chrome\Application\chromedriver.exe"
    os.environ["webdriver.chrome.driver"] = chromedriver
    driver = webdriver.Chrome(chromedriver)
    driver.get('https://www.91mobiles.com/phonefinder.php')
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    element = driver.find_element_by_css_selector("input[id*='pf4gyes']")
    driver.execute_script("arguments[0].click();", element)
    
    time.sleep(2)
    
    phone_links = []
    
    while True:
        try:
            driver.find_element_by_css_selector("span[onclick*='next']")
        except NoSuchElementException:
            break
            
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        phones = soup.find_all('div', class_='finder_snipet_wrap')
        
        for phone in phones:
            phone_links.append(f'https://www.91mobiles.com{phone.find("li").find_next()["href"]}')
            
        element = driver.find_element_by_css_selector("span[onclick*='next']")
        driver.execute_script("arguments[0].click();", element)

        time.sleep(.5+2*random.random())
        
    element = driver.find_element_by_css_selector("input[id*='pf4gyes']")
    driver.execute_script("arguments[0].click();", element)
    
    time.sleep(2)
    
    element = driver.find_element_by_css_selector("input[id*='pf5gyes']")
    driver.execute_script("arguments[0].click();", element)
    
    
    while True:
        try:
            driver.find_element_by_css_selector("span[onclick*='next']")
        except NoSuchElementException:
            break
            
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        phones = soup.find_all('div', class_='finder_snipet_wrap')
        
        for phone in phones:
            phone_links.append(f'https://www.91mobiles.com{phone.find("li").find_next()["href"]}')
            
        element = driver.find_element_by_css_selector("span[onclick*='next']")
        driver.execute_script("arguments[0].click();", element)

        time.sleep(.5+2*random.random())
        
    driver.close()
    return phone_links

The function below returns a dictionary of phones scraped from each link in the passed list

In [ ]:
def get_phones_specs(phone_links):
    index_dict = {}
    
    for ind, link in zip(range(len(phone_links)), phone_links):
        specs_dict = {}
        
        try:
            response = requests.get(link, headers={"User_Agent": UserAgent().random})
        except:
            continue
        
        soup = BeautifulSoup(response.text, 'html.parser')
        name = soup.find('h1', class_='h1_pro_head').text.split()
        price = soup.find('span', class_='big_prc', itemprop='price').text
        tables = soup.find_all('table', class_='spec_table')
        
        specs_dict['Brand'] = name[0]
        specs_dict['Model'] = ' '.join(name[1:])
        specs_dict['Price'] = price
        
        for table in tables[1:]:
            spec_ttle = table.find_all('td', class_='spec_ttle')
            spec_des = table.find_all('td', class_='spec_des')
            for title, desc in zip(spec_ttle, spec_des):
                specs_dict[title.text] = desc.text.strip()

        index_dict[ind] = specs_dict
        time.sleep(.5+2*random.random())
    
    return index_dict

In [ ]:
links = get_phones_links_4g_5g()

In [ ]:
len(links)

In [ ]:
phone_dict = get_phones_specs(links)

In [ ]:
df = pd.DataFrame(phone_dict).T